#Enterprise Fleet Analytics Pipeline: Focuses on the business outcome (analytics) and the domain (fleet/logistics).

![logistics](logistics_project.png)

Download the data from the below gdrive and upload into the catalog
https://drive.google.com/drive/folders/1J3AVJIPLP7CzT15yJIpSiWXshu1iLXKn?usp=drive_link

##**1. Data Munging** -

####1. Visibily/Manually opening the file and capture couple of data patterns (Manual Exploratory Data Analysis)

#####Create a catalog called logistics

In [0]:
%sql
create catalog if not exists logistics;

######Created a schema, volume and directory as below manually
/Volumes/logistics/logistics_schema/logistics_volume/logistics_dir/

######Keep the below files in directory

logistics_shipment_detail_3000

######Manual Exploratory Data Analysis

It is a Semi-Structured data with comma seperator (JSON)<br/>
No Header, No comments, footer is there in the data<br/>
File have list[dictioniries]<br/>
Total columns are 11<br/>
All columns has even data accross file.<br/>


####2. Programatically try to find couple of data patterns applying below EDA (File: logistics_source1)
1. Apply inferSchema and toDF to create a DF and analyse the actual data.
2. Analyse the schema, datatypes, columns etc.,
3. Analyse the duplicate records count and summary of the dataframe.

In [0]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.getOrCreate()

In [0]:
raw_source1=spark.read.csv("/Volumes/logistics/logistics_schema/logistics_volume/logistics_dir/logistics_source1",header='true',inferSchema='true')
#.toDF('shipment_id','first_name','last_name','age','role')
print("printschema")
raw_source1.printSchema()
print("summary")
display(raw_source1.summary())
print("describe")
display(raw_source1.describe())
print("count")
display(raw_source1.count())
print("sample")
display(raw_source1.sample(.1))
display(raw_source1)


In [0]:
#Important passive EDA structure functions we can use
raw_source1.printSchema()#I am realizing the id & age columns are having some non numerical values (supposed to be numeric)
print(raw_source1.columns)#I am understanding the column numbers/order and the column names
print(raw_source1.dtypes)#Realizing the datatype of every columns (even we can do programattic column & type identification for dynamic programming)
for i in raw_source1.dtypes:
    if i[1]=='string':
        print(i[0])

print(raw_source1.schema)#To identify the structure of the data in the StructType and StructField format

###a. Passive Data Munging -  (File: logistics_source1  and logistics_source2)
Without modifying the data, identify:<br>
Shipment IDs that appear in both master_v1 and master_v2<br>
Records where:<br>
1. shipment_id is non-numeric
2. age is not an integer<br>

Count rows having:
3. fewer columns than expected
4. more columns than expected

In [0]:
#Create a Spark Session Object
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.getOrCreate()

In [0]:
raw_source2=spark.read.csv("/Volumes/logistics/logistics_schema/logistics_volume/logistics_dir/logistics_source2",header='true',inferSchema='true')

In [0]:
#Important passive EDA data functions we can use
#We identified few patterns on this data
#1. Deduplication of rows and given column(s)
#2. Null values ratio across all columns
#3. Distribution (Dense) of the data across all number columns
#4. Min, Max values
#5. StdDeviation - 
#6. Percentile - Distribution percentage from 0 to 100 in 4 quadrants of 25%
print("actual count of the data source1",raw_source1.count())
print("actual count of the data source2",raw_source2.count())
print("de-duplicated record (all columns) count source1",raw_source1.distinct().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated record (all columns) count source2",raw_source2.distinct().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated record (all columns) count source1",raw_source1.dropDuplicates().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated record (all columns) count source2",raw_source2.dropDuplicates().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated given shipment_id column count source1",raw_source1.dropDuplicates(['shipment_id']).count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated given shipment_id column count source2",raw_source2.dropDuplicates(['shipment_id']).count())#de duplicate the entire columns of the given  dataframe
print("source1")
display(raw_source1.describe())
display(raw_source1.summary())

print("source2")
display(raw_source2.describe())
display(raw_source2.summary())
print(raw_source1.dtypes)
print(raw_source2.dtypes)

###**b. Active Data Munging** File: logistics_source1 and logistics_source2

#####1.Combining Data + Schema Merging (Structuring)
1. Read both files without enforcing schema
2. Align them into a single canonical schema: shipment_id,
first_name,
last_name,
age,
role,
hub_location,
vehicle_type,
data_source
3. Add data_source column with values as: system1, system2 in the respective dataframes

In [0]:
from pyspark.sql.functions import lit
struct1="shipment_id string,first_name string,last_name string,age string,role string,data_source string"
active_mung_source1=spark.read.schema(struct1).csv("/Volumes/logistics/logistics_schema/logistics_volume/logistics_dir/logistics_source1",header='true')
active_mung_source1=active_mung_source1.withColumn('data_source',lit('system1'))
struct2="shipment_id string,first_name string,last_name string,age string,role string,hub_location string,vehicle_type string,data_source string"
active_mung_source2=spark.read.schema(struct2).csv("/Volumes/logistics/logistics_schema/logistics_volume/logistics_dir/logistics_source2",header='true')
active_mung_source2=active_mung_source2.withColumn('data_source',lit('system2'))
display(active_mung_source1)
display(active_mung_source2)

merged_df=active_mung_source1.unionByName(active_mung_source2,allowMissingColumns=True)
display(merged_df)
merged_df.write.csv("/Volumes/logistics/logistics_schema/logistics_volume/logistics_dir/logistics_merged",mode="overwrite")

In [0]:
merged_df.schema

#####2. Cleansing, Scrubbing: 
Cleansing (removal of unwanted datasets)<br>
1. Mandatory Column Check - Drop any record where any of the following columns is NULL:shipment_id, role<br>
2. Name Completeness Rule - Drop records where both of the following columns are NULL: first_name, last_name<br>
3. Join Readiness Rule - Drop records where the join key is null: shipment_id<br>

Scrubbing (convert raw to tidy)<br>
4. Age Defaulting Rule - Fill NULL values in the age column with: -1<br>
5. Vehicle Type Default Rule - Fill NULL values in the vehicle_type column with: UNKNOWN<br>
6. Invalid Age Replacement - Replace the following values in age:
"ten" to -1
"" to -1<br>
7. Vehicle Type Normalization - Replace inconsistent vehicle types: 
truck to LMV
bike to TwoWheeler

In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,ShortType

struct3 = StructType([
    StructField('shipment_id', StringType(), True),
    StructField('first_name', StringType(), True),
    StructField('last_name', StringType(), True),
    StructField('age', StringType(), True),
    StructField('role', StringType(), True),
    StructField('data_source', StringType(), False),
    StructField('hub_location', StringType(), True),
    StructField('vehicle_type', StringType(), True)
])
                   
clean_raw_df=spark.read.schema(struct3).csv('/Volumes/logistics/logistics_schema/logistics_volume/logistics_dir/logistics_merged/', mode='permissive')
clean_raw_df.printSchema()
print(clean_raw_df.count())
display(clean_raw_df)

In [0]:
print("shipment_id or role is null, will get deleted")

cleanseddf=clean_raw_df.na.drop(how="any",subset=["shipment_id","role"])
display(cleanseddf)

print("firstname and lastname is null, will get deleted")
cleanseddf=cleanseddf.na.drop(how="all",subset=["first_name","last_name"])
display(cleanseddf)

print("Apply join readniess rule")
cleanseddf=cleanseddf.filter(cleanseddf["shipment_id"].isNotNull())
display(cleanseddf)

In [0]:
#Scrubbing (convert raw to tidy)
#4. Age Defaulting Rule - Fill NULL values in the age column with: -1
#5. Vehicle Type Default Rule - Fill NULL values in the vehicle_type column with: UNKNOWN
#6. Invalid Age Replacement - Replace the following values in age: "ten" to -1 "" to -1
7##. Vehicle Type Normalization - Replace inconsistent vehicle types: truck to LMV bike to TwoWheeler

#scrubbeddf1=cleanseddf.na.fill('not provided',subset=["lastname","profession"])#fill will help us replace #nulls with some value
#display(scrubbeddf1)
#find_replace_values_dict1={'Pilot':'Captain','Actor':'Celeberity'}
#find_replace_values_dict2={'not provided':'NA'}
#scrubbeddf2=scrubbeddf1.na.replace(find_replace_values_dict1,subset=["profession"])#fill function is #helping us find and replace the values
#scrubbeddf3=scrubbeddf2.na.replace(find_replace_values_dict2,subset=["lastname"])
#display(scrubbeddf3)


scrubbeddf1=cleanseddf.na.fill('-1',subset=["age"])
display(scrubbeddf1)

scrubbeddf2=scrubbeddf1.na.fill('UNKNOWN',subset=["vehicle_type"])
display(scrubbeddf2)

replace_dict1={'ten':'-1','""':'-1'}
scrubbeddf3=scrubbeddf2.na.replace(replace_dict1,subset=["age"])
display(scrubbeddf3)


replace_dict2={'Truck':'LMV','Bike':'TwoWheeler'}
scrubbeddf4=scrubbeddf3.na.replace(replace_dict2,subset=["vehicle_type"])
display(scrubbeddf4)

####3. Standardization, De-Duplication and Replacement / Deletion of Data to make it in a usable format

In [0]:
#display(scrubbeddf3.where("id in ('4000001')"))#before row level dedup
#dedupdf1=scrubbeddf3.distinct()#It will remove the row level duplicates
#display(dedupdf1.where("id in ('4000001')"))

#print("non prioritized deduplication, just remove the duplicates retaining only the first row")
#display(dedupdf1.coalesce(1).where("id in ('4000003')"))#before col level dedup
#dedupdf2=dedupdf1.coalesce(1).dropDuplicates(subset=["id"])#It will remove the column level duplicates #(retaining the first row in the dataframe)
#display(dedupdf2.where("id in ('4000003')"))
#print("prioritized deduplication based on age")
#display(dedupdf1.coalesce(1).where("id in ('4000003')"))#before col level dedup
#dedupdf1.coalesce(1).where("id in ('4000003')").orderBy(["id","age"],ascending=[True,False]).show(3)
#dedupdf2=dedupdf1.coalesce(1).orderBy(["id","age"],ascending=[True,False]).dropDuplicates(subset=["id"])#It will remove the column level duplicates (retaining the first row in the dataframe)
#display(dedupdf2.where("id in ('4000003')"))

In [0]:
display(scrubbeddf4.where("shipment_id in ('5000002')"))
deduppdf1=scrubbeddf4.distinct()
display(deduppdf1.where("shipment_id in ('5000002')"))

display(deduppdf1.coalesce(1).where("shipment_id in ('5000002')"))#before col level dedup
deduppdf2=deduppdf1.coalesce(1).dropDuplicates(subset=["shipment_id"])#It will remove the column level duplicates (retaining the first row in the dataframe
display(deduppdf2.coalesce(1).where("shipment_id in ('5000002')"))

deduppdf2=deduppdf1.coalesce(1).orderBy(["shipment_id","age"],ascending=[True,False]).dropDuplicates(subset=["shipment_id"])#It will remove the column level duplicates (retaining the first row in the dataframe)
display(deduppdf2.where("shipment_id in ('5000002')"))

Detail Dataframe creation <br>
1. Read Data from logistics_shipment_detail.json
2. As this data is a clean json data, it doesn't require any cleansing or scrubbing.

In [0]:
from pyspark.sql.types import IntegerType,FloatType,DateType,StringType,DateType,StructType,StructField
structjson=StructType([StructField("shipment_id",IntegerType(),True),
                        StructField("order_id",IntegerType(),True),
                        StructField("source_city",StringType(),True),
                        StructField("destination_city",StringType(),True),
                        StructField("shipment_status",StringType(),True),
                        StructField("cargo_type",StringType(),True),
                        StructField("vehicle_type",StringType(),True),
                        StructField("payment_mode",StringType(),True),
                        StructField("shipment_weight_kg",FloatType(),True),
                        StructField("shipment_cost",FloatType(),True),
                        StructField("shipment_date",DateType(),True)
                    ])

read_json=spark.read.schema(structjson).option("multiline",True).json("dbfs:/Volumes/logistics/logistics_schema/logistics_volume/logistics_dir/logistics_shipment_detail_3000.json")
display(read_json)

Standardizations:<br>

1. Add a column<br> 
Source File: logistics_shipment_detail_3000.json<br>: domain as 'Logistics'
2. Column Uniformity: 
role - Convert to lowercase<br>
Source File: logistics_source1 & logistics_source2<br>
vehicle_type - Convert values to UPPERCASE<br>
Source Files: logistics_shipment_detail_3000.json (and the merged master files)
hub_location - Convert values to initcap case<br>
3. Format Standardization:<br>
Source Files: logistics_shipment_detail_3000.json
Convert shipment_ref to string<br>
Pad to 10 characters with leading zeros<br>
Convert dispatch_date to yyyy-MM-dd<br>
Ensure delivery_cost has 2 decimal precision<br>
4. Data Type Standardization<br>
Standardizing column data types to fix schema drift and enable mathematical operations.<br>
Source File: logistics_source1 & logistics_source2 <br>
age: Cast String to Integer<br>
Source File: logistics_shipment_detail_3000.json<br>
shipment_weight_kg: Cast to Double<br>
Source File: logistics_shipment_detail_3000.json<br>
is_expedited: Cast to Boolean<br>
5. Naming Standardization <br>
Source File: logistics_source1 & logistics_source2<br>
Rename: first_name to staff_first_name<br>
Rename: last_name to staff_last_name<br>
Rename: hub_location to origin_hub_city<br>
6. Reordering columns logically in a better standard format:<br>
Source File: All 3 files<br>
shipment_id (Identifier), staff_first_name (Dimension)staff_last_name (Dimension), role (Dimension), origin_hub_city (Location), shipment_cost (Metric), ingestion_timestamp (Audit)

In [0]:
stand_addcol_json=read_json.withColumn("domain",lit("Logistics"))
display(stand_addcol_json)

In [0]:
#Column Uniformity: role - Convert to lowercase
#Source File: logistics_source1 & logistics_source2
#vehicle_type - Convert values to UPPERCASE
#Source Files: logistics_shipment_detail_3000.json (and the merged master files) hub_location - Convert #values to initcap case
from pyspark.sql.functions import col,lit,lower,upper,initcap
stand_uniform_source1=active_mung_source1.withColumn("role",lower(col("role")))
display(stand_uniform_source1)

stand_uniform_source2=active_mung_source2.withColumn("role",lower(col("role")))
display(stand_uniform_source2)

upcase_json=read_json.withColumn("vehicle_type",upper(col("vehicle_type")))
display(upcase_json)

initcap_csv=merged_df.withColumn("hub_location",initcap(col("hub_location")))
display(initcap_csv)

In [0]:
read_json.printSchema()

Deduplication:
1. Apply Record Level De-Duplication
2. Apply Column Level De-Duplication (Primary Key Enforcement)

##2. Data Enrichment - Detailing of data
Makes your data rich and detailed <br>

###### Adding of Columns (Data Enrichment)
*Creating new derived attributes to enhance traceability and analytical capability.*

**1. Add Audit Timestamp (`load_dt`)**
Source File: logistics_source1 and logistics_source2<br>
* **Scenario:** We need to track exactly when this record was ingested into our Data Lakehouse for auditing purposes.
* **Action:** Add a column `load_dt` using the function `current_timestamp()`.

**2. Create Full Name (`full_name`)**
Source File: logistics_source1 and logistics_source2<br>
* **Scenario:** The reporting dashboard requires a single field for the driver's name instead of separate columns.
* **Action:** Create `full_name` by concatenating `first_name` and `last_name` with a space separator.
* **Result:** "Rajesh" + " " + "Kumar" -> **"Rajesh Kumar"**

**3. Define Route Segment (`route_segment`)**
Source File: logistics_shipment_detail_3000.json<br>
* **Scenario:** The logistics team wants to analyze performance based on specific transport lanes (Source to Destination).
* **Action:** Combine `source_city` and `destination_city` with a hyphen.
* **Result:** "Chennai" + "-" + "Pune" -> **"Chennai-Pune"**

**4. Generate Vehicle Identifier (`vehicle_identifier`)**
Source File: logistics_shipment_detail_3000.json<br>
* **Scenario:** We need a unique tracking code that immediately tells us the vehicle type and the shipment ID.
* **Action:** Combine `vehicle_type` and `shipment_id` to create a composite key.
* **Result:** "Truck" + "_" + "500001" -> **"Truck_500001"**

###### Deriving of Columns (Time Intelligence)
*Extracting temporal features from dates to enable period-based analysis and reporting.*<br>
Source File: logistics_shipment_detail_3000.json<br>
**1. Derive Shipment Year (`shipment_year`)**
* **Scenario:** Management needs an annual performance report to compare growth year-over-year.
* **Action:** Extract the year component from `shipment_date`.
* **Result:** "2024-04-23" -> **2024**

**2. Derive Shipment Month (`shipment_month`)**
* **Scenario:** Analysts want to identify seasonal peaks (e.g., increased volume in December).
* **Action:** Extract the month component from `shipment_date`.
* **Result:** "2024-04-23" -> **4** (April)

**3. Flag Weekend Operations (`is_weekend`)**
* **Scenario:** The Operations team needs to track shipments handled during weekends to calculate overtime pay or analyze non-business day capacity.
* **Action:** Flag as **'True'** if the `shipment_date` falls on a Saturday or Sunday.

###### Enrichment/Business Logics (Calculated Fields)
*Deriving new metrics and financial indicators using mathematical and date-based operations.*<br>
Source File: logistics_shipment_detail_3000.json<br>

**1. Calculate Unit Cost (`cost_per_kg`)**
* **Scenario:** The Finance team wants to analyze the efficiency of shipments by determining the cost incurred per unit of weight.
* **Action:** Divide `shipment_cost` by `shipment_weight_kg`.
* **Logic:** `shipment_cost / shipment_weight_kg`

**2. Track Shipment Age (`days_since_shipment`)**
* **Scenario:** The Operations team needs to monitor how long it has been since a shipment was dispatched to identify potential delays.
* **Action:** Calculate the difference in days between the `current_date` and the `shipment_date`.
* **Logic:** `datediff(current_date(), shipment_date)`

**3. Compute Tax Liability (`tax_amount`)**
* **Scenario:** For invoicing and compliance, we must calculate the Goods and Services Tax (GST) applicable to each shipment.
* **Action:** Calculate 18% GST on the total `shipment_cost`.
* **Logic:** `shipment_cost * 0.18`

###### Remove/Eliminate (drop, select, selectExpr)
*Excluding unnecessary or redundant columns to optimize storage and privacy.*<br>
Source File: logistics_source1 and logistics_source2<br>

**1. Remove Redundant Name Columns**
* **Scenario:** Since we have already created the `full_name` column in the Enrichment step, the individual name columns are now redundant and clutter the dataset.
* **Action:** Drop the `first_name` and `last_name` columns.
* **Logic:** `df.drop("first_name", "last_name")`

##### Splitting & Merging/Melting of Columns
*Reshaping columns to extract hidden values or combine fields for better analysis.*<br>
Source File: logistics_shipment_detail_3000.json<br>
**1. Splitting (Extraction)**
*Breaking one column into multiple to isolate key information.*
* **Split Order Code:**
  * **Action:** Split `order_id` ("ORD100000") into two new columns:
    * `order_prefix` ("ORD")
    * `order_sequence` ("100000")
* **Split Date:**
  * **Action:** Split `shipment_date` into three separate columns for partitioning:
    * `ship_year` (2024)
    * `ship_month` (4)
    * `ship_day` (23)

**2. Merging (Concatenation)**
*Combining multiple columns into a single unique identifier or description.*
* **Create Route ID:**
  * **Action:** Merge `source_city` ("Chennai") and `destination_city` ("Pune") to create a descriptive route key:
    * `route_lane` ("Chennai->Pune")

## 3. Data Customization & Processing - Application of Tailored Business Specific Rules

### **UDF1: Complex Incentive Calculation**
**Scenario:** The Logistics Head wants to calculate a "Performance Bonus" for drivers based on tenure and role complexity.

**Action:** Create a Python function `calculate_bonus(role, age)` and register it as a Spark UDF.

**Logic:**
* **IF** `Role` == 'Driver' **AND** `Age` > 50:
  * `Bonus` = 15% of Salary (Reward for Seniority)
* **IF** `Role` == 'Driver' **AND** `Age` < 30:
  * `Bonus` = 5% of Salary (Encouragement for Juniors)
* **ELSE**:
  * `Bonus` = 0

**Result:** A new derived column `projected_bonus` is generated for every row in the dataset.

---

### **UDF2: PII Masking (Privacy Compliance)**
**Scenario:** For the analytics dashboard, we must hide the full identity of the staff to comply with privacy laws (GDPR/DPDP), while keeping names recognizable for internal managers.

**Business Rule:** Show the first 2 letters, mask the middle characters with `****`, and show the last letter.

**Action:** Create a UDF `mask_identity(name)`.

**Example:**
* **Input:** `"Rajesh"`
* **Output:** `"Ra****h"`
<br>
**Note: Convert the above udf logic to inbult function based transformation to ensure the performance is improved.**

## 4. Data Core Curation & Processing (Pre-Wrangling)
*Applying business logic to focus, filter, and summarize data before final analysis.*

**1. Select (Projection)**<br>
Source Files: logistics_source1 and logistics_source2<br>
* **Scenario:** The Driver App team only needs location data, not sensitive HR info.
* **Action:** Select only `first_name`, `role`, and `hub_location`.

**2. Filter (Selection)**<br>
Source File: json<br>
* **Scenario:** We need a report on active operational problems.
* **Action:** Filter rows where `shipment_status` is **'DELAYED'** or **'RETURNED'**.
* **Scenario:** Insurance audit for senior staff.
* **Action:** Filter rows where `age > 50`.

**3. Derive Flags & Columns (Business Logic)**<br>
Source File: json<br>
* **Scenario:** Identify high-value shipments for security tracking.
* **Action:** Create flag `is_high_value` = **True** if `shipment_cost > 50,000`.
* **Scenario:** Flag weekend operations for overtime calculation.
* **Action:** Create flag `is_weekend` = **True** if day is Saturday or Sunday.

**4. Format (Standardization)**<br>
Source File: json<br>
* **Scenario:** Finance requires readable currency formats.
* **Action:** Format `shipment_cost` to string like **"₹30,695.80"**.
* **Scenario:** Standardize city names for reporting.
* **Action:** Format `source_city` to Uppercase (e.g., "chennai" → **"CHENNAI"**).

**5. Group & Aggregate (Summarization)**<br>
Source Files: logistics_source1 and logistics_source2<br>
* **Scenario:** Regional staffing analysis.
* **Action:** Group by `hub_location` and **Count** the number of staff.
* **Scenario:** Fleet capacity analysis.
* **Action:** Group by `vehicle_type` and **Sum** the `shipment_weight_kg`.

**6. Sorting (Ordering)**<br>
Source File: json<br>
* **Scenario:** Prioritize the most expensive shipments.
* **Action:** Sort by `shipment_cost` in **Descending** order.
* **Scenario:** Organize daily dispatch schedule.
* **Action:** Sort by `shipment_date` (Ascending) then `priority_flag` (Descending).

**7. Limit (Top-N Analysis)**<br>
Source File: json<br>
* **Scenario:** Dashboard snapshot of critical delays.
* **Action:** Filter for 'DELAYED', Sort by Cost, and **Limit to top 10** rows.

## 5. Data Wrangling - Transformation & Analytics
*Combining, modeling, and analyzing data to answer complex business questions.*

### **1. Joins**
Source Files:<br>
Left Side (staff_df):<br> logistics_source1 & logistics_source2<br>
Right Side (shipments_df):<br> logistics_shipment_detail_3000.json<br>
#### **1.1 Frequently Used Simple Joins (Inner, Left)**
* **Inner Join (Performance Analysis):**
  * **Scenario:** We only want to analyze *completed work*. Connect Staff to the Shipments they handled.
  * **Action:** Join `staff_df` and `shipments_df` on `shipment_id`.
  * **Result:** Returns only rows where a staff member is assigned to a valid shipment.
* **Left Join (Idle Resource check):**
  * **Scenario:** Find out which staff members are currently *idle* (not assigned to any shipment).
  * **Action:** Join `staff_df` (Left) with `shipments_df` (Right) on `shipment_id`. Filter where `shipments_df.shipment_id` is NULL.

#### **1.2 Infrequent Simple Joins (Self, Right, Full, Cartesian)**
* **Self Join (Peer Finding):**
  * **Scenario:** Find all pairs of employees working in the same `hub_location`.
  * **Action:** Join `staff_df` to itself on `hub_location`, filtering where `staff_id_A != staff_id_B`.
* **Right Join (Orphan Data Check):**
  * **Scenario:** Identify shipments in the system that have *no valid driver* assigned (Data Integrity Issue).
  * **Action:** Join `staff_df` (Left) with `shipments_df` (Right). Focus on NULLs on the left side.
* **Full Outer Join (Reconciliation):**
  * **Scenario:** A complete audit to find *both* idle drivers AND unassigned shipments in one view.
  * **Action:** Perform a Full Outer Join on `shipment_id`.
* **Cartesian/Cross Join (Capacity Planning):**
  * **Scenario:** Generate a schedule of *every possible* driver assignment to *every* pending shipment to run an optimization algorithm.
  * **Action:** Cross Join `drivers_df` and `pending_shipments_df`.

#### **1.3 Advanced Joins (Semi and Anti)**
* **Left Semi Join (Existence Check):**
  * **Scenario:** "Show me the details of Drivers who have *at least one* shipment." (Standard filtering).
  * **Action:** `staff_df.join(shipments_df, "shipment_id", "left_semi")`.
  * **Benefit:** Performance optimization; it stops scanning the right table once a match is found.
* **Left Anti Join (Negation Check):**
  * **Scenario:** "Show me the details of Drivers who have *never* touched a shipment."
  * **Action:** `staff_df.join(shipments_df, "shipment_id", "left_anti")`.

### **2. Lookup**<br>
Source File: logistics_source1 and logistics_source2 (merged into Staff DF)<br>
* **Scenario:** Validation. Check if the `hub_location` in the staff file exists in the corporate `Master_City_List`.
* **Action:** Compare values against a reference list.

### **3. Lookup & Enrichment**<br>
Source File: logistics_source1 and logistics_source2 (merged into Staff DF)<br>
* **Scenario:** Geo-Tagging.
* **Action:** Lookup `hub_location` ("Pune") in a Master Latitude/Longitude table and enrich the dataset by adding `lat` and `long` columns for map plotting.

### **4. Schema Modeling (Denormalization)**<br>
Source Files: All 3 Files (logistics_source1, logistics_source2, logistics_shipment_detail_3000.json)<br>
* **Scenario:** Creating a "Gold Layer" Table for PowerBI/Tableau.
* **Action:** Flatten the Star Schema. Join `Staff`, `Shipments`, and `Vehicle_Master` into one wide table (`wide_shipment_history`) so analysts don't have to perform joins during reporting.

### **5. Windowing (Ranking & Trends)**<br>
Source Files:<br>
logistics_source2: Provides hub_location (Partition Key).<br>
logistics_shipment_detail_3000.json: Provides shipment_cost (Ordering Key)<br>
* **Scenario:** "Who are the Top 3 Drivers by Cost in *each* Hub?"
* **Action:**
  1. Partition by `hub_location`.
  2. Order by `total_shipment_cost` Descending.
  3. Apply `dense_rank()` and `row_number()
  4. Filter where `rank or row_number <= 3`.

### **6. Analytical Functions (Lead/Lag)**<br>
Source File: <br>
logistics_shipment_detail_3000.json<br>
* **Scenario:** Idle Time Analysis.
* **Action:** For each driver, calculate the days elapsed since their *previous* shipment.

### **7. Set Operations**<br>
Source Files: logistics_source1 and logistics_source2<br>
* **Union:** Combining `Source1` (Legacy) and `Source2` (Modern) into one dataset (Already done in Active Munging).
* **Intersect:** Identifying Staff IDs that appear in *both* Source 1 and Source 2 (Duplicate/Migration Check).
* **Except (Difference):** Identifying Staff IDs present in Source 2 but *missing* from Source 1 (New Hires).

### **8. Grouping & Aggregations (Advanced)**<br>
Source Files:<br>
logistics_source2: Provides hub_location and vehicle_type (Grouping Dimensions).<br>
logistics_shipment_detail_3000.json: Provides shipment_cost (Aggregation Metric).<br>
* **Scenario:** The CFO wants a subtotal report at multiple levels:
  1. Total Cost by Hub.
  2. Total Cost by Hub AND Vehicle Type.
  3. Grand Total.
* **Action:** Use `cube("hub_location", "vehicle_type")` or `rollup()` to generate all these subtotals in a single query.

##6. Data Persistance (LOAD)-> Data Publishing & Consumption<br>

Store the inner joined, lookup and enrichment, Schema Modeling, windowing, analytical functions, set operations, grouping and aggregation data into the delta tables.

##7.Take the copy of the above notebook and try to write the equivalent SQL for which ever applicable.